In [1]:
import random
import hashlib

In [2]:
# y^2 = x^3 + a*x + b

In [3]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
    
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
    
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
    
    is_on_curve((x3, y3), p)
        
    return x3, y3

def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
    
def apply_double_and_add_method(G, k, p):
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point, p)
        
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
    
    is_on_curve(target_point, p)
    
    return target_point

In [4]:
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [5]:
is_on_curve(G, p)

# Alice generates her private and public key pair

In [6]:
# private key of Alice
d = random.getrandbits(256)

# public key of Alice
Q = apply_double_and_add_method(G = G, k = d, p = p)

In [45]:
d

13433848042164743573906504080612824839915133641008164720264532415569497568864

In [7]:
# random_key = random.getrandbits(256)
random_point = apply_double_and_add_method(G = G, k = random_key, p = p)

In [15]:
random_key

2549423989004030740645076493356421339477309545835940582037060479260904522953

In [16]:
# message = b"ECC beats RSA"
message = b"ECC beats Diffie-Hellman, too!"
hash_hex = hashlib.sha1(message).hexdigest()
hash_int = int(hash_hex, 16)

In [17]:
hash_int

531204124611006148816545464087066627965678629689

# Sign

In [18]:
# (r, s)
r = ( random_point[0] ) % n
s = ( ( hash_int + r * d ) * pow(random_key, -1, n) ) % n

In [19]:
r

76058233759865599401027122649904720385258380809806134291104208040832864904366

In [20]:
s

8856213573513779368811935123774923867490066008725770383186210144604547226738

# Verification

In [ ]:
w = pow(s, -1, n)
u1 = apply_double_and_add_method(G = G, k = ( hash_int * w ) % n, p = p)
u2 = apply_double_and_add_method(G = Q, k = ( r * w ) % n, p = p)

# u1 + u2
checkpoint = add_points(P = u1, Q = u2, p = p)

assert checkpoint[0] == r

# Attacking

If 2 messages were signed with same random key, an attacker can extract sender's private key!

In [ ]:
# s1 = ( h1 + r * d ) * random_key ^ -1
# s2 = ( h2 + r * d ) * random_key ^ -1
# s1 - s2 = ( h1 + r * d ) * random_key ^ -1 - ( h2 + r * d ) * random_key ^ -1
# s1 - s2 = random_key ^ -1 * (h1 + r * d - h2 - r * d )
# s1 - s2 =  random_key ^ -1 * (h1 - h2)
# (s1 - s2) / (h1 - h2) = random_key ^ -1
# (h1 - h2) / (s1 - s2) = random_key

In [23]:
s1 = 8044708637677890244949153424877160784554741168781389939176486503430395752522
h1 = 320026739459778556085970613903841025917693204146

s2 = 8856213573513779368811935123774923867490066008725770383186210144604547226738
h2 = 531204124611006148816545464087066627965678629689

In [31]:
random_key_prime = ( ( (h1 - h2) % n ) * pow(s1 - s2, -1, n) ) % n

In [32]:
random_key_prime

2549423989004030740645076493356421339477309545835940582037060479260904522953

In [33]:
assert random_key == random_key_prime

In [36]:
random_point_prime = apply_double_and_add_method(G = G, k = random_key_prime, p = p)
r_prime = random_point_prime[0]

In [39]:
assert r_prime == r

In [ ]:
# s1 = ( h1 + r * d ) * random_key ^ -1
# s1 * random_key = h1 + r * d
# s1 * random_key - h1 = r * d
# (s1 * random_key - h1) / r = d

In [43]:
d_prime = ( (s1 * random_key_prime - h1) * pow(r, -1, n) ) % n

In [44]:
d_prime

13433848042164743573906504080612824839915133641008164720264532415569497568864

In [46]:
assert d_prime == d